In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv('data2.csv',sep = ' ',names = ['x1','x2','Y'])
df.head()

,x1,x2,Y
0,0.01,0.049979,1
1,0.02,0.099833,1
2,0.03,0.149438,1
3,0.04,0.198669,1
4,0.05,0.247404,1


In [ ]:
px.scatter(df,x ='x1',y = 'x2',color = 'Y')

Реалізувати алгоритм зворотного розповсюдження похибки.
Вимоги:
1. - [x] Алгоритм повинен приймати на вхід:
     - Кількість шарів
     - Кількість нейронів на кожному шарі
     - Активаційну функцію на кожному шарі
     - Крок градієнтного спуску \alpha
2. - [x] Алгоритм повинен бути оформленим у вигляді класу або функції
3. Алгоритм повинен підтримувати наступні активаційні функці:
    - ReLu
    - Sigmoid
    - Tanh
    - Softmax для останнього шару
4. - [x] Повинна бути можливість дотренувати алгоритм з іншим \alpha
5. - [x] Алгоритм повинен видавати значення цільової функції
6. - [x] Реалізуйте можливість намалювати області класифікації для алгоритму

- Точність алгоритму має бути близькою до 100%. Розбивати вибірку на тренувальну та тестову не потрібно (лише для цього завдання!).

За бажанням можна реалізувати mini-batch gradient descent, RMSProp або Ada

In [ ]:
class Layer():
  def __init__(self):
    self.input = None
    self.output = None


  def forward_propagation(self):
    pass


  def backward_propagation(self):
    pass


In [ ]:
## second attempt

class FCLayer(Layer):
  def __init__(self,input_size,output_size):
    self.w = np.random.rand(input_size,output_size) - 0.5
    self.b = np.random.rand(1,output_size) - 0.5



  def forward_propagation(self,input):
    self.input = input.reshape(1,self.w.shape[0])
    self.output = self.input.dot(self.w) + self.b
    return self.output


  def backward_propagation(self,error,alpha):
    input_error = np.dot(error, self.w.T)
    weights_error = np.dot(self.input.T, error)

    self.w -= alpha * weights_error
    self.b -= alpha * error

    return input_error

class ActivationLayer(Layer):
  def __init__(self,act,de_act):
    self.act = act
    self.de_act = de_act

  def forward_propagation(self,input):
    self.input = input
    self.output = self.act(self.input)
    return self.output

  def backward_propagation(self,error,alpha):

    return self.act(self.input)*error


In [ ]:
class Network():
  def __init__(self):
    self.layers = []
    self.loss = None
    self.loss_prime = None

  def add_layear(self,layer):
    self.layers.append(layer)

  def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

  def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []
        input_data  = input_data
        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result


  def fit(self,x_train,y_train,epochs,alpha=0.01):
    samples = x_train.shape[0]
    x_train = x_train.to_numpy()
    err = 0

    for i in range(epochs):

      for j in range(samples):

        output = x_train[j]
       # print('fit: ', output)
        for layer in self.layers:
          output = layer.forward_propagation(output)
          #print(layer," ",output)
        err += self.loss(y_train[j], output)

        error = self.loss_prime(y_train[j], output)
        for layer in reversed(self.layers):
          error = layer.backward_propagation(error, alpha)

      err /= samples
      if int(i/100)==i/100:
        print(f"Epoch {i} {err}")




In [ ]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;


def cross_entropy_two_classes(y_true,y_pred):
  return -(y_true*np.log(y_pred+1e-7)+(1-y_true)*np.log(1-y_pred+1e-7))

def cross_entropy_prime_two_classes(y_true,y_pred):
  return (y_pred-y_true)/(y_pred*(1-y_pred)+0.01)

In [ ]:
## Пункт 3
def relu(x):
  return np.maximum(x,0)

def sigmoid(x):
  #x = np.float128(x)
  return 1/(1+np.exp(-1*x))

def tanh(x):
  x = np.float128(x)
  return (np.exp(2*x)-1)/(np.exp(2*x)+1)

def softmax(x):
  #x = np.float128(x)
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum(axis=0)


def relu_prime(x):
  return  np.where(x>0,1,0)

def sigmoid_prime(x):
  return sigmoid(x)*(1-sigmoid(x))

def tanh_prime(x):
  return 1-np.tanh(x)**2;


In [ ]:
def calculate_decision_boundary_mass(min_x1,max_x1,min_x2,max_x2,model):
  inc1 = (max_x1-min_x1)/100
  inc2 = (max_x2-min_x2)/100
  x1grid = np.arange(min_x1-0.1,max_x1+0.1, inc1)
  x2grid = np.arange(min_x2-0.1,max_x2+0.1, inc2)
  xx, yy = np.meshgrid(x1grid, x2grid)
  r1, r2 = xx.flatten(), yy.flatten()
  r11, r22 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))
  grid = np.hstack((r11,r22))

  Y = np.array(net.predict(grid)).reshape(len(r1),1)
  Z = np.where(Y<0.42,0,1)
  return r1,r2,Y,Z.flatten()

In [ ]:
net = Network()

net.use(cross_entropy_two_classes,cross_entropy_prime_two_classes)

net.add_layear(FCLayer(input_size = 2,output_size = 50))
net.add_layear(ActivationLayer(act = relu,de_act = relu_prime))
net.add_layear(FCLayer(input_size = 50,output_size = 50))
net.add_layear(ActivationLayer(act = relu,de_act = relu_prime))
net.add_layear(FCLayer(input_size = 50,output_size = 2))
net.add_layear(ActivationLayer(act = relu,de_act = relu_prime))
net.add_layear(FCLayer(input_size = 2,output_size = 1))
net.add_layear(ActivationLayer(act = sigmoid ,de_act = sigmoid_prime))

In [ ]:
net.fit(df[['x1','x2']],df['Y'],epochs = 901,alpha = 0.01)

Epoch 0 [[0.48999088]]
Epoch 100 [[0.68341161]]
Epoch 200 [[0.68341161]]
Epoch 300 [[0.68341161]]


KeyboardInterrupt: ignored

Тренувати цю модель було дуже важко адже в будь-якій незрозумілій ситуації воно знаходило точку з нульовою похідною( зачасту це предіктився лише один клас і після цього воно майже не змінювало значення валідаційної функції. Сотні спроб зробити з mse не увінчались успіхом, тому я спробував це зробити через кросс-ентропію. Але і це не допомогло. Після я взяв інші дані, що також не допомогло. Також я використовував різні комбінації шарів, що також не увіналось успіхом з різними поєднаннями активаційних функцій. Нижче я ще спробував це зробити через keras і в мене знову ж таки нічого не вийшло.

In [ ]:
desX1,desX2,Z,desY = calculate_decision_boundary_mass(df['x1'].min(), df['x1'].max(),df['x2'].min(),df['x2'].max(),net)
boundries_df = pd.DataFrame({"x1":desX1,"x2":desX2})

fig = go.Figure()
fig.add_trace(go.Scatter(x = desX1,y = desX2,mode = 'markers',marker =dict(color =  desY,colorscale = 'Sunsetdark')))

fig.add_trace(go.Scatter(x =  df['x1'],y
                         = df['x2'],mode = 'markers',marker =dict(color = df['Y'],colorscale = 'Turbo') ))
fig.show()

In [ ]:
import keras

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(3,input_dim=2,activation = 'relu'))
model.add(keras.layers.Dense(3,activation = 'relu'))
model.add(keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',metrics = ['accuracy'],optimizer='adam')

history = model.fit(df[['x1','x2']],df['Y'],epochs=100)


In [ ]:
Z[Z>0.42].shape

(6004,)

In [ ]:
Z.shape

(12285, 1)

In [ ]:
desX1,desX2,Z,desY = calculate_decision_boundary_mass(df['x1'].min(), df['x1'].max(),df['x2'].min(),df['x2'].max(),model)
boundries_df = pd.DataFrame({"x1":desX1,"x2":desX2})

fig = go.Figure()
fig.add_trace(go.Scatter(x = desX1,y = desX2,mode = 'markers',marker =dict(color =  desY,colorscale = 'Sunsetdark')))

fig.add_trace(go.Scatter(x =  df['x1'],y
                         = df['x2'],mode = 'markers',marker =dict(color = df['Y'],colorscale = 'Turbo') ))
fig.show()

Я був надихнутий [статтею](https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65)